In [12]:
import torch
from torch import Tensor
import numpy as np
import math

In [2]:
torch.set_grad_enabled( False )

## Class definition

In [7]:
class Module ( object ) :
    def __init__(self):
        super(Model, self).__init__()
        
        
    def forward ( self , * input ):
        a = x.mm(w)
        s = ReLu(a)
        
        
        
        
    def backward ( self , * gradwrtoutput ):
        grad_w = 2* (y_pred - y)
        w = w - lr * grad_w
    
        
    def param ( self ):
        return []
    
    
    def sgd():
        batch = next_training_batch(data, self.batch_size)
        L = LossMSE(y_pred,y)    

In [71]:
class Linear(object):
    def __init__(self, in_features, out_features):
        super(Linear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = torch.Tensor(size=(self.in_features,self.out_features ))
        
        print(self.weight)
    
        

In [88]:
Sequential(args = (Linear(3,3),Linear(3,3),Linear(3,3)))

tensor([[-1.8521e+24,  5.6472e-43,  2.7708e+20],
        [ 2.2945e+02,  1.9791e+05,  3.9586e+12],
        [ 1.0333e+18,  0.0000e+00,  0.0000e+00]])
tensor([[-2.9445e+25,  5.6472e-43,  1.3816e+31],
        [ 1.9203e+31,  6.3015e+34,  6.2688e+22],
        [ 4.7428e+30,  1.5975e-43,  0.0000e+00]])
tensor([[-1.8521e+24,  5.6472e-43,  1.4312e+13],
        [ 7.2156e+22,  7.0975e+22,  1.3032e-43],
        [ 2.1019e-44,  0.0000e+00,  0.0000e+00]])
{'args': (<__main__.Linear object at 0x00000193823DE6A0>, <__main__.Linear object at 0x00000193823D6550>, <__main__.Linear object at 0x00000193824216A0>)}


In [73]:
class Tanh(object):
    def __init__(self, ):
        object.__init__(self)
        
    def do(self,x):
        y = (math.exp(x) - math.exp(-x)) / ( math.exp(x) +  math.exp(-x))
        return y

In [86]:
class Sequential():
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        print(kwargs)
        
    

In [ ]:
class LossMSE(object):
    def __init__(self, ):
        object.__init__(self)
        loss = (y_pred - y).pow(2).sum()

In [18]:
class ReLu(object):
    
    def __init__(self, ):
        object.__init__(self)
        
    def do(self,x):
        y = x.clamp(min = 0)
        return y


## Sample generation

In [14]:
def generate_disc_set(nb):
    input = Tensor(nb, 2).uniform_(-1, 1)
    target = input.pow(2).sum(1).sub(2 / math.pi).sign().add(1).div(2).long()
    return input, target

train_input, train_target = generate_disc_set(1000)
test_input, test_target = generate_disc_set(1000)

mean, std = train_input.mean(), train_input.std()

train_input.sub_(mean).div_(std)
test_input.sub_(mean).div_(std)



mini_batch_size = 100

tensor([4., 5.])
